# Let's try the ChefGPT

Let's try our newly trained model and see how well it performs

In [ ]:
# We are going to use the same T5 wrapper used in previous labs, only this time, we are going to load the trained
# model by referencing the `fine_tuned_t5_recipes_base_5k_v1` folder (which is where we saved the trained model in the previous lab).
# We do the same for the tokenizer.

from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained("fine_tuned_t5_recipes_base_5k_v1")
tokenizer = T5Tokenizer.from_pretrained("fine_tuned_t5_recipes_base_5k_v1")

Let's setup our food items

In [13]:
# Specify some food items which will be used in our recipe prompt

food_items = "chicken, peas, carrots, cream of chicken soup, cream of celery soup, chicken broth, milk, Bisquick mix"

For more information about the different settings visit: https://huggingface.co/blog/how-to-generate

In [14]:
# Define a function which accepts a food items list as text, builds a prompt and calls the `generate` function on the model to generate the recipe.
# There are several ways to use the `generate` functions and we are going to try a few of those in this lab.
# If you would like to learn more about the various options, please refer to:
# [https://huggingface.co/blog/how-togenerate](https://huggingface.co/blog/how-togenerate).

def generate_recipe_v1(prompt):
    input_text = f"generate recipe: {prompt}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output_ids = model.generate(input_ids, max_length=512, num_return_sequences=1)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

In [15]:
# Call our function to generate a recipe using the food items list declared in the second cell, and print the result

recipe = generate_recipe_v1(food_items)
s = recipe.replace("ingredients:", "\n\nIngredients:").replace("directions:", "\n\nDirections:")
print(s)

title: Chicken Soup 

Ingredients: 1 lb. chicken, 1 c. peas, 1 c. carrots, 1 c. cream of chicken soup, 1 c. cream of celery soup, 1 c. chicken broth, 1 c. milk, 1 c. Bisquick mix 

Directions: Cook chicken in boiling water. Add peas and carrots. Add cream of chicken soup and celery soup. Add chicken broth milk and Bisquick mix. Stir until mixture is thickened. Add chicken broth and milk. Stir in Bisquick mix. Pour into a greased 9 x 13-inch baking dish. Bake at 350u00b0 for 20 minutes.


In [16]:
# A variation on the code in cell above where we use different parameters to control the output of the `generate` function.

generation_kwargs_v3 = {
    "max_length": 512,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.95,
    "num_return_sequences": 3
}

def generate_recipe_v3(prompt):
    input_text = f"generate recipe: {prompt}"
    input_ids = tokenizer(input_text, return_tensors="pt", padding='max_length', truncation=True, max_length=512).input_ids
    output_ids = model.generate(input_ids, **generation_kwargs_v3)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

recipe = generate_recipe_v3(food_items)
s = recipe.replace("ingredients:", "\n\nIngredients:").replace("directions:", "\n\nDirections:")
print(s)

title: Chicken Bisquick 

Ingredients: 1 small cube (120 g) chicken stock, drained, 1 can chopped peas, peeled and cooked, 1 lb diced carrots, 1 box cream of chicken soup, 2 cups cream of celery soup, 1 can chicken broth, 2 cups milk, 1/2 cup Bisquick mix 

Directions: Mix all ingredients well. Put in a large skillet to cook. Remove from heat. Add the chicken and peas and cook at 350u00b0. Add the soup. Boil about 3 minutes. Add all ingredients and cook for about 5 minutes. Remove from heat and bake at 400u00b0 for about 30 minutes stirring occasionally. Reduce heat and stir occasionally stirring occasionally.


In [17]:
# Another variation on the code in cell above where we use different parameters to control the output of the `generate` function.

generation_kwargs_v4 = {
    "max_length": 512,
    "min_length": 64,
    "no_repeat_ngram_size": 3,
    "do_sample": True,
    "top_k": 60,
    "top_p": 0.95
}

def generate_recipe_v4(prompt):
    input_text = f"generate recipe: {prompt}"
    inputs = tokenizer(
        input_text, 
        max_length=512, 
        padding="max_length", 
        truncation=True, 
        return_tensors="pt"
    )

    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    output_ids = model.generate(
        input_ids=input_ids, 
        attention_mask=attention_mask,
        **generation_kwargs_v4
    )
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

recipe = generate_recipe_v4(food_items)
s = recipe.replace("ingredients:", "\n\nIngredients:").replace("directions:", "\n\nDirections:")
print(s)

title: Chicken Tomato Soup 

Ingredients: 2 lbs. chicken, chopped, 2 lb. peas, chopped carrots, 1 c. cream of chicken soup, 1 small can cream of celery soup, 4 cups chicken broth, 1 cups milk, 1 1/2 cups Bisquick mix 

Directions: Cook chicken with 1 large can pea salad (not brown), 2 cups white beans, cooked, 1/2 cup carrots and soup, and cover with hot water to cook.


## Using Pipelines

In [ ]:
# In this cell we showcase the use of the `pipeline` wrapper of the `transformers` library.
# We load the tuned model by specifying the `fine_tuned_t5_recipes_base_5k_v1` folder where the tuned model was saved.
# In the call to the `pipeline` function, we specify `text2text-generation` as the value to the `task` parameter since T5 is a sequence to sequence LLM.

from transformers import pipeline
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained("fine_tuned_t5_recipes_base_5k_v1")
tokenizer = T5Tokenizer.from_pretrained("fine_tuned_t5_recipes_base_5k_v1")

generator = pipeline(model=model, tokenizer=tokenizer, task="text2text-generation")

In [20]:
# In the eighth cell we specify a food items list, set values for some of the possible parameters and use the `generator` function with our prompt and parameters.

food_items = "chicken, peas, carrots, cream of chicken soup, cream of celery soup, chicken broth, milk, Bisquick mix"

tokenizer_kwargs = {
    "max_length": 512,
    "do_sample": True,
    "top_k": 60,
    "top_p": 0.95,
    "num_return_sequences": 2
}

response = generator(f"generate recipe: {food_items}", **tokenizer_kwargs)

print(response[0]['generated_text'].replace("ingredients:", "\n\nIngredients:").replace("directions:", "\n\nDirections:"))

title: Chicken Mash 

Ingredients: 1 can chicken, 1 can peas, 1 can carrots, diced, 1 can cream of chicken soup, 1 can cream of celery soup, 1 can chicken broth, 1 lb. milk, 3/4 cup Bisquick mix 

Directions: In a small bowl combine the soup and water to make a thicker soup. Mix the two and mix until thick. Pour over chicken. Cook until chicken is tender. Drain. Remove from water and cool. Add the rest of the liquid (except cheese) and let stand 5 minutes.
